# Reproduction

In this notebook we're just going to verify the results of Going For Three and Choking Under the Pressure, which trained logistic regression models on kicks from 2000 to 2011 and 2017, respectively. Initially we use all kickers, not just those that had had at least 50 kicks at the time of the attempt.

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import glm as glm_sm
from sklearn.model_selection import train_test_split
import numpy as np
import mysql.connector
import itertools
from scipy.stats import chi2

Error: Invalid response: 500 Internal Server Error

Set up a connection to the mysql database served locally.

In [ ]:
cnx = mysql.connector.connect(user='root', password='mOntie20!mysql', host='127.0.0.1', database='nfl')

Error: Invalid response: 500 Internal Server Error

Our base query doesn't change between models so we'll keep it here.

In [ ]:
base_query = '''select
p.pid,fg.good,fg.dist, 
g.seas as year, k.seas as seasons,
case when g.temp<50 then 1 else 0 end as cold,
case when g.stad like "%Mile High%" then 1 else 0 end as altitude,
case when g.humd>=60 then 1 else 0 end as humid,
case when g.wspd>=10 then 1 else 0 end as windy,
case when g.v=p.off then 1 else 0 end as away_game,
case when g.wk>=10 then 1 else 0 end as postseason,
case when (pp.qtr=p.qtr) and ((pp.timd-p.timd)>0 or (pp.timo-p.timo)>0) then 1 else 0 end as iced,
case g.surf when 'Grass' then 0 else 1 end as turf,
case when g.cond like "%Snow%" then 1 when g.cond like "%Rain%" and not "Chance Rain" then 1 else 0 end as precipitation,
case when p.qtr=4 and ABS(p.ptso - p.ptsd)>21 then 0
when p.qtr=4 and p.min<2 and ABS(p.ptso - p.ptsd)>8 then 0
when p.qtr=4 and p.min<2 and p.ptso-p.ptsd < -7 then 0
when p.qtr<=3 then 0
when p.qtr=4 and p.min>=2 and ABS(p.ptso - p.ptsd)<21 then 0
when p.qtr=4 and p.min<2 and p.ptso-p.ptsd >=5 and p.ptso-p.ptsd <=8 then 0
when p.qtr=4 and p.min<2 and p.ptso-p.ptsd >=-4 and p.ptso-p.ptsd <=-6 then 0
else 1 end as pressure'''

## Going For Three

Query the database for the data used by Going For Three. i.e. pre-2011

In [ ]:
query = base_query+'''
from FGXP fg
left join PLAY p on fg.pid=p.pid
left join game g on p.gid=g.gid
join kicker k on k.player = fg.fkicker and g.gid=k.gid
join PLAY pp on pp.pid=p.pid-1 and pp.gid=p.gid
where fg.fgxp='FG' -- not an xp
and g.seas <= 2011
order by p.pid
'''

df = pd.read_sql(query, cnx, index_col = 'pid')
df.head(10)

Going For Three didn't use the year and seasons variables in their tabulated results, so our first model won't either.

In [ ]:
model = glm_sm('good ~ ' + '+'.join(df.drop(['year','seasons','good'], axis=1).columns.values), df, family=sm.families.Binomial())
result = model.fit(method='newton')
print(result.summary())
base_ll = pd.read_html(result.summary().tables[0].as_html())[0].iloc[4,3]

We now add back in the year and seasons of experience and control for kickers that dont make it in the NFL (so >50 kicks).

In [ ]:
query = base_query+'''
from FGXP fg
left join PLAY p on fg.pid=p.pid
left join game g on p.gid=g.gid
join kicker k on k.player = fg.fkicker and g.gid=k.gid
join PLAY pp on pp.pid=p.pid-1 and pp.gid=p.gid
where fg.fgxp='FG' -- not an xp
and fg.fkicker in (
select fkicker
from fifty) -- has had at least 50 attempts overall
and fg.pid > (
select pid
from fifty
where fg.fkicker = fkicker) -- this kick came after the 50th attempt
and g.seas <= 2011
order by p.pid
'''

df = pd.read_sql(query, cnx, index_col = 'pid')
df.head(5)

In [ ]:
model = glm_sm('good ~ '+'+'.join(df.drop(['good'], axis=1).columns.values), df, family=sm.families.Binomial())
result = model.fit(method='newton')
print(result.summary())

## Choking Under the Pressure

In Choking Under the Pressure, they used similar data now from 2000-2017.
Lets repeat the modelling with this data, again leaving out the seasons and year covariates and not controlling for >50 kicks

In [ ]:
query = base_query+'''
from FGXP fg
left join PLAY p on fg.pid=p.pid
left join game g on p.gid=g.gid
join kicker k on k.player = fg.fkicker and g.gid=k.gid
join PLAY pp on pp.pid=p.pid-1 and pp.gid=p.gid
where fg.fgxp='FG' -- not an xp
and g.seas <= 2017
order by p.pid
'''

df = pd.read_sql(query, cnx, index_col = 'pid')
df.head(5)

In [ ]:
model = glm_sm('good ~ '+'+'.join(df.drop(['good','seasons','year'], axis=1).columns.values), df, family=sm.families.Binomial())
result = model.fit(method='newton')
print(result.summary())